In [46]:
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType
import torch
from datasets import load_dataset
import os
from torch.utils.data import DataLoader, Dataset
from tqdm import tqdm

device = "cuda"
model_name_or_path = "bigscience/bloomz-560m"
tokenizer_name_or_path = "bigscience/bloomz-560m"
peft_config = PromptTuningConfig(
    task_type=TaskType.CAUSAL_LM,
    prompt_tuning_init=PromptTuningInit.TEXT,
    num_virtual_tokens=8,
    tokenizer_name_or_path=model_name_or_path,
)

dataset_name = "judge_data"
checkpoint_name = f"{dataset_name}_{model_name_or_path}_{peft_config.peft_type}_{peft_config.task_type}_v1.pt".replace(
    "/", "_"
)
text_column = "prompt"
label_column = "text_label"
max_length = 64
lr = 3e-2
num_epochs = 50
batch_size = 8

In [49]:
import pickle
with open('train_json.pickle', 'rb') as f:
    dataset = pickle.load(f)

dataset["root"][0]


{'ID': 'TRAIN_0001',
 'prompt': ' facts: Ramon Nelson was riding his bike when he suffered a lethal blow to the back of his head with a baseball bat. After two eyewitnesses identified Lawrence Owens from an array of photos and then a lineup, he was tried and convicted for Nelson’s death. Because Nelson was carrying cocaine and crack cocaine potentially for distribution, the judge at Owens’ bench trial ruled that Owens was probably also a drug dealer and was trying to “knock [Nelson] off.” Owens was found guilty of first-degree murder and sentenced to 25 years in prison.\nOwens filed a petition for a writ of habeas corpus on the grounds that his constitutional right to due process was violated during the trial. He argued that the eyewitness identification should have been inadmissible based on unreliability and that the judge impermissibly inferred a motive when a motive was not an element of the offense. The district court denied the writ of habeas corpus, and Owens appealed. The U.S. 

In [27]:
from collections import Counter

labels = list(map(lambda x: x['Label'], dataset['train']))
counter = Counter(labels)
counter

Counter({2: 33, 1: 17})

In [28]:
labels = list(map(lambda x: x['Label'], dataset['test']))
counter = Counter(labels)
counter

Counter({0: 3399})

In [29]:
classes = [k.replace("_", " ") for k in dataset["train"].features["Label"].names]
dataset = dataset.map(
    lambda x: {"text_label": [classes[label] for label in x["Label"]]},
    batched=True,
    num_proc=1,
)



Loading cached processed dataset at /root/.cache/huggingface/datasets/ought___raft/twitter_complaints/1.1.0/79c4de1312c1e3730043f7db07179c914f48403101f7124e2fe336f6f54d9f84/cache-d412a834430f1a60.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/ought___raft/twitter_complaints/1.1.0/79c4de1312c1e3730043f7db07179c914f48403101f7124e2fe336f6f54d9f84/cache-53af518f9692b496.arrow


In [30]:
tokenizer = AutoTokenizer.from_pretrained(model_name_or_path)
print(tokenizer.pad_token_id )
print(tokenizer.eos_token_id)
if tokenizer.pad_token_id is None:
    tokenizer.pad_token_id = tokenizer.eos_token_id
target_max_length = max([len(tokenizer(class_label)["input_ids"]) for class_label in classes])
print(target_max_length)


3
2
3


In [34]:
def preprocess_function(examples):
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id] # pad token 더해줌 eos 토큰으로 사용
#         print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids # "... label :"의 토큰 뒤에 라벨 토큰 까지 더해줌
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids # "... label :"의 토큰까지 -100으로 가려주는 듯
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        # 패딩토큰이 신기하게 앞에서부터 붙는다.
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        
        #패딩 토큰 길이만큼 마스킹해줌
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        
        # input까지 -100 토큰으로 만듦
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [35]:
# inputs = [f"{text_column} : {x} Label : " for x in dataset['train'][text_column]]
# targets = [str(x) for x in dataset['train'][label_column]]

# model_inputs = tokenizer(inputs)
# labels = tokenizer(targets)

# labels['input_ids'][0] + [tokenizer.pad_token_id]

In [36]:
processed_datasets = dataset.map(
    preprocess_function,
    batched=True,
    num_proc=1,
    remove_columns=dataset["train"].column_names,
    load_from_cache_file=False,
    desc="Running tokenizer on dataset",
)

Running tokenizer on dataset:   0%|          | 0/50 [00:00<?, ? examples/s]

Running tokenizer on dataset:   0%|          | 0/3399 [00:00<?, ? examples/s]

In [39]:
train_dataset = processed_datasets["train"]
eval_dataset = processed_datasets["train"]


train_dataloader = DataLoader(
    train_dataset, shuffle=True, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True
)
eval_dataloader = DataLoader(eval_dataset, collate_fn=default_data_collator, batch_size=batch_size, pin_memory=True)

In [37]:
model = AutoModelForCausalLM.from_pretrained(model_name_or_path)
model = get_peft_model(model, peft_config)
print(model.print_trainable_parameters())

trainable params: 8192 || all params: 559222784 || trainable%: 0.0014648902430985358
None


In [40]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=0,
    num_training_steps=(len(train_dataloader) * num_epochs),
)

In [41]:
model = model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(train_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(eval_dataloader)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(eval_dataloader)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(train_dataloader)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

100%|██████████| 7/7 [00:00<00:00, 17.89it/s]


epoch=0: train_ppl=tensor(4.2881e+14, device='cuda:0') train_epoch_loss=tensor(33.6920, device='cuda:0') eval_ppl=tensor(173441.0312, device='cuda:0') eval_epoch_loss=tensor(12.0636, device='cuda:0')


100%|██████████| 7/7 [00:00<00:00, 17.76it/s]


epoch=1: train_ppl=tensor(50299.2578, device='cuda:0') train_epoch_loss=tensor(10.8257, device='cuda:0') eval_ppl=tensor(8842.6875, device='cuda:0') eval_epoch_loss=tensor(9.0873, device='cuda:0')


100%|██████████| 7/7 [00:00<00:00, 17.73it/s]


...

epoch=47: train_ppl=tensor(6.8145, device='cuda:0') train_epoch_loss=tensor(1.9190, device='cuda:0') eval_ppl=tensor(6.6551, device='cuda:0') eval_epoch_loss=tensor(1.8954, device='cuda:0')


100%|██████████| 7/7 [00:00<00:00, 17.71it/s]


epoch=48: train_ppl=tensor(6.6115, device='cuda:0') train_epoch_loss=tensor(1.8888, device='cuda:0') eval_ppl=tensor(6.5413, device='cuda:0') eval_epoch_loss=tensor(1.8781, device='cuda:0')


100%|██████████| 7/7 [00:00<00:00, 17.60it/s]

epoch=49: train_ppl=tensor(6.2774, device='cuda:0') train_epoch_loss=tensor(1.8370, device='cuda:0') eval_ppl=tensor(6.5237, device='cuda:0') eval_epoch_loss=tensor(1.8754, device='cuda:0')


In [42]:
from peft import PeftModel, PeftConfig

peft_model_id = "stevhliu/bloomz-560m_PROMPT_TUNING_CAUSAL_LM"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

In [44]:
inputs = tokenizer(
    f'{text_column} : {"@nationalgridus I have no water and the bill is current and paid. Can you do something about this?"} Label : ',
    return_tensors="pt",
)

In [45]:
model.to(device)

with torch.no_grad():
    inputs = {k: v.to(device) for k, v in inputs.items()}
    outputs = model.generate(
        input_ids=inputs["input_ids"], attention_mask=inputs["attention_mask"], max_new_tokens=10, eos_token_id=3
    )
    print(tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True))

['Tweet text : @nationalgridus I have no water and the bill is current and paid. Can you do something about this? Label : complaint']


['Tweet text : @nationalgridus I have no water and the bill is current and paid. Can you do something about this? Label : complaint']